# Genetic Algoritm - 8 Queens Problem

# Author: Pedro Malandrin Klesse

# Steps
1. Initial Population
2. Selection
3. Crossover and Mutation
4. Returns to Step 2 until the iterations are over

In [103]:
# Libraries

import random
import numpy as np
import matplotlib.pyplot as plt


In [97]:
# Generating the initial population randomly

num_ind = 500
DIMENSION = 8
initial_population = np.zeros((num_ind, DIMENSION), dtype=int)

for i in range(num_ind):
  for j in range(DIMENSION):
    initial_population[i][j] = random.randint(0,DIMENSION-1)

# Prints the initial population
"""for i in range(num_ind):
  print(initial_population[i])"""


'for i in range(num_ind):\n  print(initial_population[i])'

In [25]:
# The fitness function returns the number of pairs that queens are not attacking each other minus the collisions
# So, if a queen enters in a collision situation, 28 (max) decreases 1, and so on

def fitness(individual):
  collisions = 0
  ind_len = len(individual)
  for i in range(ind_len-1):
    for j in range(i+1, ind_len):
      # Collision in lines
      if individual[i] == individual[j]:
        collisions += 1
      # Collision in diagonals
      elif abs(individual[j]-individual[i]) == abs(i-j):
        collisions += 1
  return 28 - collisions

# This functions return the fitness for a population

def return_fitness(population):
  fit = []
  for ind in population:
    f = fitness(ind)
    fit.append(f)
  return fit



In [26]:
# The chosen selection method is the tournament selection

def tournament_selection(fitnesses):
  ind1 = -1
  ind2 = -1

  while ind1 == ind2:
    # Tournament 1
    chosen = random.sample(range(0, num_ind),2)
    if fitnesses[chosen[0]] > fitnesses[chosen[1]]:
      ind1 = chosen[0]
    else:
      ind1 = chosen[1]
    
    # Tournament 2
    chosen = random.sample(range(0, num_ind),2)
    if fitnesses[chosen[0]] > fitnesses[chosen[1]]:
      ind2 = chosen[0]
    else:
      ind2 = chosen[1]

    return ind1, ind2



In [73]:
def crossover_one_point(ids, population):

  point = random.randint(1, 7)

  father1 = population[ids[0]]
  father2 = population[ids[1]]

  offspring1 = np.concatenate([father1[:point], father2[point:]])
  offspring2 = np.concatenate([father2[:point], father1[point:]])

  return offspring1, offspring2


def crossover_two_point(ids, population):
  point2 = -1
  point1 = random.randint(1,7)

  father1 = population[ids[0]]
  father2 = population[ids[1]]

  while point2 != point1:
    point2 = random.randint(1,7)

  offspring1 = np.concatenate([father1[point1:], father2[point1:point2], father1[:point2]])
  offspring2 = np.concatenate([father2[point1:], father1[point1:point2], father2[:point2]])

  return offspring1, offspring2


def crossover_uniform(ids, population):
  father1 = population[ids[0]]
  father2 = population[ids[1]]
  offspring1 = np.zeros([8])
  offspring2 = np.zeros([8])
  for i in range (len(father1)):
    if i % 2 == 0:
      offspring1[i] = father1[i]
      offspring2[i] = father2[i]
    else:
      offspring1[i] = father2[i]
      offspring2[i] = father1[i]
  return offspring1, offspring2

# Combining randomly the three crossover functions
def crossover_mixed(ids, population):
  num = random.randint(0, 2)
  if num == 0:
    offspring1, offspring2 = crossover_one_point(ids, population)
  elif num == 1:
    offspring1, offspring2 = crossover_two_point(ids, population)
  else:
    offspring1, offspring2 = crossover_uniform(ids, population)
  return offspring1, offspring2



In [28]:
def elitism(fitnesses):
  id1 = fitnesses.index(max(fitnesses))

  fitnesses.pop(id1)

  id2 = fitnesses.index(max(fitnesses))

  return id1, id2

In [29]:
def mutation(sons, rate):
  
  for i in range(len(sons)):
    if random.random() < rate:
        index = random.randint(0,DIMENSION-1)
        sons[i][index] = random.randint(0,DIMENSION-1)

  return sons

In [129]:
num_generations = 100

# ideal_fitness = 28
#best_fit_time = np.zeros((num_generations))

for it in range(num_generations):
  new_population = np.zeros((num_ind,8),dtype=int)
  fit = return_fitness(initial_population)

  best_id = fit.index(max(fit))
  #best_fit_time[it] = max(fit)

  elit = elitism(fit.copy())
  new_population[0] = new_population[elit[0]]
  new_population[1] = new_population[elit[1]]

  num_offsprings = 2
  while num_offsprings < num_ind:

    # Tournament Selection
    win_ind = tournament_selection(fit)

    #Crossover
    offsprings = crossover_mixed(win_ind, initial_population)

    # Mutation rate
    mut_rate = 0.08

    # Mutation with rate of 8%
    offsprings = mutation(offsprings,mut_rate)

    # Insert offsprings in the new population
    new_population[num_offsprings] = offsprings[0]
    new_population[num_offsprings+1] = offsprings[1]

    # Increase offsprings number
    num_offsprings += 2

  # Replace the old population to the new one
  new_population = new_population.copy()

fit = return_fitness(new_population)
best_id = fit.index(max(fit))
print("Fit of the best chromosome (max is 28): " + str(max(fit)))
print("Best chromosome: " + str(new_population[best_id]))




Fit of the best chromosome (max is 28): 26
Best chromosome: [5 3 0 3 7 4 1 1]
